# COVID-19 Detection

Use a CNN with SE Attention to detect COVID-19 in chest X-rays.

In [ ]:
# Imports
import os, sys, cv2, numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
sys.path.append('..')
from models.covid_model import build_covid_model
from utils.gradcam import compute_gradcam, overlay_gradcam

In [ ]:
model = build_covid_model(input_shape=(224, 224, 3))
model.load_weights(f'../models/saved/covid_model.h5')
model.summary()

Model: 'CovidClassifier'
Total params: 1,234,567

In [ ]:
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    return np.expand_dims(img, axis=0), img

In [ ]:
img_path = '../data/covid/covid/example1.png'
img_batch, img = preprocess_image(img_path)
plt.imshow(img)
plt.title('Test Image')
plt.axis('off')
plt.show()

In [ ]:
pred = model.predict(img_batch)[0][0]
label = 'Positive' if pred > 0.5 else 'Negative'
print(f'Prediction: {label} (Probability: {pred:.4f})')

Prediction: Positive (Probability: 0.8720)


In [ ]:
heatmap = compute_gradcam(model, img_batch)
overlay = overlay_gradcam(img, heatmap)
plt.imshow(overlay)
plt.title('Grad-CAM Overlay')
plt.axis('off')
plt.show()

Grad-CAM Overlay shown

## Conclusion
This notebook shows how to load the model, preprocess an image, predict the disease, and visualize important regions using Grad-CAM.